# Network Analysis for ABCD data

http://dx.plos.org/10.1371/journal.pbio.1002328  
https://www.sciencedirect.com/science/article/pii/S105381191730109X?via%3Dihub

In [ ]:
# ! pip install python-louvain

In [1]:
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import community
from sklearn.metrics.cluster import normalized_mutual_info_score
import pickle
import pdb
import statistics
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj
from visbrain.io import download_file

/Users/gracer/.local/lib/python3.6/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


## Data of interest (from R notebook)

### Read in labels

In [2]:
labels = pd.read_csv('/Users/gracer/Google Drive/ABCD/important_txt/locations.csv', sep=",")

In [3]:
interest =  pd.read_csv('/Users/gracer/Google Drive/ABCD/important_txt/data4analysis.txt', sep=" ", header=None, 
                 index_col=False)
subs=interest[0]

## Get data

In [4]:
data = glob.glob('/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-*/keep/sub-NDAR*_ses-baselineYear1Arm1_task-rest_run-0*_bold_brain_norm_zr_matrix.csv')

In [5]:
data[0:10]

['/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINVKCU1HRMA/keep/sub-NDARINVKCU1HRMA_ses-baselineYear1Arm1_task-rest_run-01_bold_brain_norm_zr_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINVKCU1HRMA/keep/sub-NDARINVKCU1HRMA_ses-baselineYear1Arm1_task-rest_run-02_bold_brain_norm_zr_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-02_bold_brain_norm_zr_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-03_bold_brain_norm_zr_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV20CNYBBC/keep/sub-NDARINV20CNYBBC_ses-baselineYear1Arm1_task-rest_run-01_bold_brain_norm_zr_matrix.csv',
 '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINVJ67B8F0J/keep/sub-N

#### Make a dictonary to store the path and subject ID

In [6]:
my_dict={}
for item in data:
    name=item.split('/')[8]
    my_dict.setdefault(name, []).append(item)

### Make a dictonary of all the file paths common to the subject list 

In [7]:
path_dict = {k: my_dict[k] for k in subs if k in my_dict}

### Make a dictonary to read in the files

In [8]:
path_dict

{'sub-NDARINV019DXLU4': ['/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV019DXLU4/keep/sub-NDARINV019DXLU4_ses-baselineYear1Arm1_task-rest_run-02_bold_brain_norm_zr_matrix.csv',
  '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV019DXLU4/keep/sub-NDARINV019DXLU4_ses-baselineYear1Arm1_task-rest_run-03_bold_brain_norm_zr_matrix.csv',
  '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV019DXLU4/keep/sub-NDARINV019DXLU4_ses-baselineYear1Arm1_task-rest_run-04_bold_brain_norm_zr_matrix.csv',
  '/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV019DXLU4/keep/sub-NDARINV019DXLU4_ses-baselineYear1Arm1_task-rest_run-01_bold_brain_norm_zr_matrix.csv'],
 'sub-NDARINV030W95VP': ['/Users/gracer/Google Drive/ABCD/ABCD_puberty/_data/ABCDworking/sub-NDARINV030W95VP/keep/sub-NDARINV030W95VP_ses-baselineYear1Arm1_task-rest_run-01_bold_brain_norm_zr_matrix.csv',
  '/Users/gracer/Google Drive/ABCD/ABCD_pub

In [9]:
data_dict={}
num=[]
for key, value in path_dict.items():
    print(len(value))
    num.append(len(value))
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, header=None,index_col=False))
#         data_dict[key]= pd.read_csv(i, header=None,index_col=False)

4
4
4
2
4
4
3
4
4
3
2
4
4
3
4
4
4
4
4
2
4
4
4
3
4
4
4
4
3
3
4
4
4
3
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
3
4
3
3
4
4
4
4
4
4
5
4
4
4
3
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
4
4
4
4
4
4
4
4
4
3
2
4
4
2
3
3
4
4
3
4
4
4
4
4
4
3


In [10]:
np.mean(num)

3.783333333333333

In [11]:
len(list(data_dict.keys()))

120

### Function to create a graph with positive or negative values and minimum correlation value

In [12]:
def create_corr_network_5(G, corr_direction, min_correlation):
    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)


### Function to make a graph object BY SUBJECT
This will return: 
* The edges (noramlized R correlation matrix, in pandas dataframe)
* The correlations (absolute value of the edges in a numpy dataframe)
* The mean_FC (the mean functional connectivity per subject/run)
* The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)

In [13]:
GRAPHS.keys()

NameError: name 'GRAPHS' is not defined

In [14]:
interest.head()

,0,1,2,3
0,sub-NDARINVMFVA1BZF,F,latepubertal,Obese
1,sub-NDARINV019DXLU4,F,latepubertal,Obese
2,sub-NDARINVT472A3DT,F,latepubertal,Obese
3,sub-NDARINV4D917H14,F,latepubertal,Obese
4,sub-NDARINVD9L81NY5,F,latepubertal,Obese


In [15]:
interest_dict=interest.set_index(interest[0]).to_dict()
def removekey(d, key):
    r = dict(d)
    del r[key]
    return r
cov_dict=removekey(interest_dict, 0)
cov_dict['sex'] = cov_dict.pop(1)
cov_dict['PCS'] = cov_dict.pop(2)
cov_dict['OVOB'] = cov_dict.pop(3)
print(cov_dict.keys())

dict_keys(['sex', 'PCS', 'OVOB'])


In [ ]:
# cov_dict['sex']['sub-NDARINVMFVA1BZF']

In [ ]:
for key in cov_dict.keys():
    for subkey, value in cov_dict[key].items():
        if subkey in GRAPHS['graphs']:
            GRAPHS['graphs'][subkey].append(value)
            print(GRAPHS['graphs'][subkey])

### Differences in mean FC

In [ ]:
list(GRAPHS['mean_FC'].keys())

In [ ]:
GRAPHS['mean_FC']['sub-NDARINVULZMADF1']

In [ ]:
# tmp=[]
# o=[]

# for key,value in GRAPHS['mean_FC'].items():
#     o=value
#     z=statistics.stdev([i for i in o if isinstance(i, int) or isinstance(i, float)])
#     x=sum([i for i in o if isinstance(i, int) or isinstance(i, float)])
#     y=len([i for i in o if isinstance(i, int) or isinstance(i, float)])
#     mu=x/y
#     o.append(mu)
#     o.append(z)

### How to delete items in list that you accidentally made

In [ ]:
# for key,value in article['mean_FC'].items():
#     del value[-7:]

In [ ]:
GRAPHS.keys()

In [ ]:
BIGdf=GRAPHS['BIGdf']

### Getting the standard deviation 

In [ ]:
# for key,value in GRAPHS['mean_FC'].items():
#     o=value[:-4]
#     try:
#         x=statistics.stdev([i for i in o if isinstance(i, int) or isinstance(i, float)])
#         print(x)
#         value.append(x)
#     except:
#         x='NaN'
#         print(x)
#         value.append(x)

In [ ]:
GRAPHS['mean_FC']['sub-NDARINVULZMADF1']

### Making dataframe to use 

In [ ]:
PID = list(GRAPHS['mean_FC'].keys())
len(PID)

In [ ]:
# list(GRAPHS['mean_FC'].values())

In [ ]:
sex=[]
PCS=[]
OVOB=[]
mean_FC=[]
sd_FC=[]
for item in list(GRAPHS['mean_FC'].values()):
    sex.append(item[-5])
    PCS.append(item[-4])
    OVOB.append(item[-3])
    mean_FC.append(item[-2])
    sd_FC.append(item[-1])

In [ ]:
len(OVOB)

### Creating a dataframe 

In [ ]:
FCmat=np.column_stack([PID, sex, PCS, OVOB, mean_FC, sd_FC])
FCdf=pd.DataFrame(FCmat, columns = ['subjects','sex','PCS','OVOB','mean','SD'])
FCdf=FCdf.fillna(0)

In [ ]:
FCdf.describe()

### Testing differences in mean FC
See the matching R notebook
Summary:
* No difference in the FC between PCS (no need to control for it in models)
* Trending difference in the FC between OVOB (may need to control for it in the model)

### Differences in modularity by subject (probs won't use, don't know what it means)

In [ ]:
GRAPHS['graphs']['sub-NDARINVULZMADF1'][10]

In [ ]:
PID = []
Parts = []
for key, value in GRAPHS['graphs'].items():
    for item in value:
        if isinstance(item,dict):
            print(key)
            print(item)
            PID.append(key)
            Parts.append(value[1])

In [ ]:
len(Parts)

In [ ]:
ROIs=[]
Modules=[]
for item in Parts:
#     print(item.keys())
    ROIs.append(list(item.keys()))
    Modules.append(list(item.values()))

In [ ]:
PIDar = np.array(PID)
ROIar=np.array(ROIs)
ModuleAr=np.array(Modules)
print(ModuleAr.shape)
print(PIDar.shape)

In [ ]:
PARTdf = pd.DataFrame(ModuleAr)
PARTdf['subjects'] = PIDar
# PARTdf

In [ ]:
# PARTdf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/PARTS.csv", sep=',', index=False)

In [ ]:
BIGdf = pd.merge(PARTdf, FCdf, on="subjects")
BIGdf.head()

In [ ]:
categories = BIGdf[['subjects', 'sex','PCS','OVOB']]
categories.head()

In [ ]:
x=BIGdf[1].value_counts().idxmax()

In [ ]:
maxes={}
for sub in PID:
    subset=BIGdf.loc[(BIGdf['subjects'] == sub)]
    for col in subset:
        macks = subset[col].value_counts().idxmax()
        maxes.setdefault(sub, []).append(macks)
    #     print(PARTdf[col].value_counts().idxmax())

In [ ]:
subjects=[]
max_vals=[]
for key, value in maxes.items():
#     print(item.keys())
    subjects.append(key)
    max_vals.append(list(value))

In [ ]:
Subar = np.array(subjects)
Valar=np.array(max_vals)
print(Subar.shape)
print(Valar.shape)

In [ ]:
MAXdf = pd.DataFrame(Valar)
MAXdf['subjects'] = Subar

In [ ]:
# MAXdf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/MAX.csv", sep=',', index=False)

In [ ]:
norm_max=[]
subset=MAXdf.loc[(MAXdf[267] == 'Normalweight')]
for col in subset:
    mechs = subset[col].value_counts().idxmax()
    norm_max.append(mechs)
#     print(PARTdf[col].value_counts().idxmax())

In [ ]:
ov_max=[]
subset=MAXdf.loc[(MAXdf[267] == 'Overweight')]
for col in subset:
    mechs = subset[col].value_counts().idxmax()
    ov_max.append(mechs)
#     print(PARTdf[col].value_counts().idxmax())

In [ ]:
ob_max=[]
subset=MAXdf.loc[(MAXdf[267] == 'Obese')]
for col in subset:
    mechs = subset[col].value_counts().idxmax()
    ob_max.append(mechs)

In [ ]:
normAR = np.array(norm_max)
ovAR = np.array(ov_max)
obAR = np.array(ob_max)

print(normAR.shape)
print(ovAR.shape)
print(obAR.shape)

In [ ]:
len(ob_max)

In [ ]:
g=community.induced_graph(partition,H)
create_corr_network_1(G=g)

### Normalized information score  
sklearn.metrics.normalized_mutual_info_score(labels_true [group 1], labels_pred [group 2], average_method=’warn’)

In [ ]:
diff_no_ov=normalized_mutual_info_score(norm_max, ov_max)

In [ ]:
diff_no_ob=normalized_mutual_info_score(norm_max, ob_max)

In [ ]:
diff_ov_ob=normalized_mutual_info_score(ov_max, ob_max)

In [ ]:
diff_no_ob

In [ ]:
diff_no_ov

In [ ]:
diff_ov_ob

##  Create a Mega graph by OVOB and PCS

In [ ]:
def make_total_graphs(dict_o_data):
    mylist=[]
    for key, val_list in dict_o_data.items():
#         print(key)
        for i in val_list:
            cor_matrix = np.asarray(i)
            mylist.append(cor_matrix)
    x=np.stack(mylist, axis=2)
    mu=np.mean(x, axis=(2))
    return(mu)

In [ ]:
GRAPHS.keys()

In [ ]:
list(GRAPHS['correlations'].values())[1]

## Creating subsets of the correlations matrices per PCS and BMI group

In [ ]:
no_subset=categories.loc[(categories['OVOB'] == 'Normalweight')]
no_dict = {k: GRAPHS['correlations'][k] for k in no_subset['subjects'] if k in GRAPHS['correlations']}

ov_subset=categories.loc[(categories['OVOB'] == 'Overweight')]
ov_dict = {k: GRAPHS['correlations'][k] for k in ov_subset['subjects'] if k in GRAPHS['correlations']}

ob_subset=categories.loc[(categories['OVOB'] == 'Obese')]
ob_dict = {k: GRAPHS['correlations'][k] for k in ob_subset['subjects'] if k in GRAPHS['correlations']}


In [ ]:
for x in np.unique(categories['PCS']):
    subset=categories.loc[(categories['PCS'] == x)]
    early_dict = {k: GRAPHS['correlations'][k] for k in early_subset['subjects'] if k in GRAPHS['correlations']}

In [ ]:
early_subset=categories.loc[(categories['PCS'] == 'earlypubertal')]
early_dict = {k: GRAPHS['correlations'][k] for k in early_subset['subjects'] if k in GRAPHS['correlations']}

late_subset=categories.loc[(categories['PCS'] == 'latepubertal')]
late_dict = {k: GRAPHS['correlations'][k] for k in late_subset['subjects'] if k in GRAPHS['correlations']}

mid_subset=categories.loc[(categories['PCS'] == 'midpubertal')]
mid_dict = {k: GRAPHS['correlations'][k] for k in mid_subset['subjects'] if k in GRAPHS['correlations']}


In [ ]:
mean_no=make_total_graphs(no_dict)
mean_ov=make_total_graphs(ov_dict)
mean_ob=make_total_graphs(ob_dict)

In [ ]:
mean_early=make_total_graphs(early_dict)
mean_late=make_total_graphs(late_dict)
mean_mid=make_total_graphs(mid_dict)

## Creates graph using the data of the correlation matrix

In [ ]:
noG = nx.from_numpy_matrix(mean_no)
ovG = nx.from_numpy_matrix(mean_ov)
obG = nx.from_numpy_matrix(mean_ob)
OVOBgraphs = [noG, ovG, obG]

In [ ]:
earlyG = nx.from_numpy_matrix(mean_early)
midG = nx.from_numpy_matrix(mean_mid)
lateG = nx.from_numpy_matrix(mean_late)
PCSgraphs = [earlyG, midG, lateG]

In [ ]:
for graph in OVOBgraphs:
    for i, nlrow in labels.iterrows():
        graph.node[i].update(nlrow[0:].to_dict())

In [ ]:
for graph in PCSgraphs:
    for i, nlrow in labels.iterrows():
        graph.node[i].update(nlrow[0:].to_dict())

In [ ]:
edges,weights = zip(*nx.get_edge_attributes(ovG,'weight').items())

In [ ]:
early_edges, early_weights = zip(*nx.get_edge_attributes(earlyG,'weight').items())
mid_edges,mid_weights = zip(*nx.get_edge_attributes(midG,'weight').items())
late_edges,late_weights = zip(*nx.get_edge_attributes(lateG,'weight').items())

In [ ]:
for graph in OVOBgraphs:
    d = nx.degree(graph)
    list(d)[0]
    nodelist, node_sizes = zip(*list(d))

In [ ]:
for graph in PCSgraphs:
    d = nx.degree(graph)
    list(d)[0]
    nodelist, node_sizes = zip(*list(d))

## Function to create network

In [ ]:
def create_corr_network(G, corr_direction, min_correlation):
    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
                
    
    #crates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(H,'weight').items())
    
    ### increases the value of weights, so that they are more visible in the graph
    weights = tuple([(1+abs(x))**1 for x in weights])
    
    #####calculates the degree of each node
    d = nx.degree(H)
    #####creates list of nodes and a list their degrees that will be used later for their sizes
    nodelist, node_sizes = zip(*list(d))
    return(H)

In [ ]:
def create_corr_network_1(G):
    #crates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

    #positions
    positions=nx.circular_layout(G)
    
    #Figure size
    plt.figure(figsize=(15,15))

    #draws nodes
    nx.draw_networkx_nodes(G,positions,node_color='#DA70D6',
                           node_size=500,alpha=0.8)
    
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, 
                            font_family='sans-serif')
        
    #draws the edges
    nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid')
    
    # displays the graph without axis
    plt.axis('off')
    #saves image
    plt.savefig("part1.png", format="PNG")
    plt.show() 

#create_corr_network_1(G)

In [ ]:
noH=create_corr_network(noG, "positive",0)
ovH=create_corr_network(ovG, "positive", 0)
obH=create_corr_network(obG, "positive", 0)
saveme={}
saveme['noH']=noH
saveme['ovH']=ovH
saveme['obH']=obH

In [ ]:
earlyH=create_corr_network(earlyG, "positive",0)
midH=create_corr_network(midG, "positive", 0)
lateH=create_corr_network(lateG, "positive", 0)
saveme={}
saveme['earlyH']=earlyH
saveme['midH']=midH
saveme['lateH']=lateH

## Getting partition values with the group graphs

In [ ]:
noPART = community.best_partition(noH)
ovPART = community.best_partition(ovH)
obPART = community.best_partition(obH)

# noPART.values()

In [ ]:
earlyPART = community.best_partition(earlyH)
midPART = community.best_partition(midH)
latePART = community.best_partition(lateH)

# noPART.values()

## Checking the normalized mutual disribution information

In [ ]:
diff_no_ov=normalized_mutual_info_score(list(noPART.values()), list(ovPART.values()))
print(diff_no_ov)

In [ ]:
diff_no_ob=normalized_mutual_info_score(list(noPART.values()), list(obPART.values()))
print(diff_no_ob)

In [ ]:
diff_early_mid=normalized_mutual_info_score(list(earlyPART.values()), list(midPART.values()))
print(diff_early_mid)

In [ ]:
diff_early_late=normalized_mutual_info_score(list(earlyPART.values()), list(latePART.values()))
print(diff_early_late)

### Summary so far
Larger difference between the normal weight and obese compared to the normal weight and the overweight

## Vizualization

In [ ]:
color_dic = {0:'blue',1:'red',2:'green',3:'purple',4:'yellow',5:'pink',6:'black',7:'orange', 8:'cyan'}

#### BMI groups

In [ ]:
PARTS=[noPART, ovPART ,obPART]
for item in PARTS:
    for key, value in item.items():
        item[key]=(color_dic[value])
#     print(item)

In [ ]:
for key, value in noPART.items():
    noH.node(data=True)[key].update({'color':value})

In [ ]:
size = float(len(set(noPART.values())))
print(size)
pos = nx.spring_layout(noH)
count = 0.
for com in set(noPART.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in noPART.keys()
                                if noPART[nodes] == com]
    nx.draw_networkx_nodes(noH, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(noH, pos, alpha=0.1)
plt.show()

In [ ]:
for key, value in ovPART.items():
    ovH.node(data=True)[key].update({'color':value})

In [ ]:
for key, value in obPART.items():
    obH.node(data=True)[key].update({'color':value})

In [ ]:
no_edge_weights = [e[2]['weight'] for e in noH.edges(data=True)]
ov_edge_weights = [e[2]['weight'] for e in ovH.edges(data=True)]
ob_edge_weights = [e[2]['weight'] for e in obH.edges(data=True)]


In [ ]:
# Define node positions data structure (dict) for plotting
no_node_colors = {node[0]: (node[1]['color']) for node in noH.nodes(data=True)}
ov_node_colors = {node[0]: (node[1]['color']) for node in ovH.nodes(data=True)}
ob_node_colors = {node[0]: (node[1]['color']) for node in obH.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
#dict(list(node_colors.items())[0:5])
#print(G.node(data=True))
no_list_colors=list(no_node_colors.values())
ov_list_colors=list(ov_node_colors.values())
ob_list_colors=list(ob_node_colors.values())

In [ ]:
no_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in noH.nodes(data=True)}
ov_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in ovH.nodes(data=True)}
ob_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in obH.nodes(data=True)}

In [ ]:
# Define node positions data structure (dict) for plotting
# node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in H.nodes(data=True)}
# node_colors = {node[0]: (node[1]['color']) for node in H.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
#dict(list(node_colors.items())[0:5])
#print(G.node(data=True))
# list_colors=list(node_colors.values())

#### PCS groups

In [ ]:
PARTS=[earlyPART, midPART ,latePART]
for item in PARTS:
    for key, value in item.items():
        item[key]=(color_dic[value])
#     print(item)

In [ ]:
for key, value in earlyPART.items():
    earlyH.node(data=True)[key].update({'color':value})

In [ ]:
earlyH.nodes(data=True)

In [ ]:
size = float(len(set(earlyPART.values())))
print(size)
pos = nx.spring_layout(earlyH)
count = 0.
for com in set(earlyPART.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in earlyPART.keys()
                                if earlyPART[nodes] == com]
    nx.draw_networkx_nodes(earlyH, pos, list_nodes, node_size = 20,
                                node_color = str(count / size))


nx.draw_networkx_edges(earlyH, pos, alpha=0.1)
plt.show()

In [ ]:
for key, value in midPART.items():
    midH.node(data=True)[key].update({'color':value})

In [ ]:
# midH.nodes(data=True)

In [ ]:
for key, value in latePART.items():
    lateH.node(data=True)[key].update({'color':value})

In [ ]:
early_edge_weights = [e[2]['weight'] for e in earlyH.edges(data=True)]
mid_edge_weights = [e[2]['weight'] for e in midH.edges(data=True)]
late_edge_weights = [e[2]['weight'] for e in lateH.edges(data=True)]

In [ ]:
# Define node positions data structure (dict) for plotting
early_node_colors = {node[0]: (node[1]['color']) for node in earlyH.nodes(data=True)}
mid_node_colors = {node[0]: (node[1]['color']) for node in midH.nodes(data=True)}
late_node_colors = {node[0]: (node[1]['color']) for node in lateH.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
#dict(list(node_colors.items())[0:5])
#print(G.node(data=True))
early_list_colors=list(early_node_colors.values())
mid_list_colors=list(mid_node_colors.values())
late_list_colors=list(late_node_colors.values())

In [ ]:
early_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in earlyH.nodes(data=True)}
mid_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in midH.nodes(data=True)}
late_node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in lateH.nodes(data=True)}

In [ ]:
# Define node positions data structure (dict) for plotting
node_positions = {node[0]: (node[1]['X'], -node[1]['Y']) for node in H.nodes(data=True)}
node_colors = {node[0]: (node[1]['color']) for node in H.nodes(data=True)}

# Preview of node_positions with a bit of hack (there is no head/slice method for dictionaries).
#dict(list(node_colors.items())[0:5])
#print(G.node(data=True))
list_colors=list(node_colors.values())

### Visualize differences

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(noH, pos=no_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = no_list_colors)
plt.title('normal', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(earlyH, pos=early_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = early_list_colors)
plt.title('early', size=15)
plt.show()

In [ ]:
early_list_colors

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(obH, pos=ob_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = ob_list_colors)
plt.title('obese', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(midH, pos=mid_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = mid_list_colors)
plt.title('Mid', size=15)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
nx.draw(lateH, pos=late_node_positions,  node_size=20,  edge_color = "lightgrey", node_color = late_list_colors)
plt.title('Late', size=15)
plt.show()

In [ ]:
nog=community.induced_graph(noPART,noH)
create_corr_network_1(G=nog)

In [ ]:
ovg=community.induced_graph(ovPART,ovH)
create_corr_network_1(G=ovg)

In [ ]:
obg=community.induced_graph(obPART,obH)
create_corr_network_1(G=obg)

In [ ]:
earlyg=community.induced_graph(earlyPART,earlyH)
create_corr_network_1(G=earlyg)

In [ ]:
midg=community.induced_graph(midPART,midH)
create_corr_network_1(G=midg)

In [ ]:
lateg=community.induced_graph(latePART,lateH)
create_corr_network_1(G=lateg)

### Summmary of results so far... 
Looks like the obese have one less module compared to the overweight and the normal weight 
Need to assess what each node is comprised of  


In [ ]:
# Coloring method
color_by = 'strength'
# Because we don't want to plot every connections, we only keep connections
# above .7
select = edges > .7
# Define the connectivity object
c_default = ConnectObj('default', nodes, edges, select=select, line_width=2.,
                       cmap='Spectral_r', color_by=color_by)
# Then, we define the sources
s_obj = SourceObj('sources', nodes, color='#ab4642', radius_min=15.)
sc.add_to_subplot(c_default, title='Color by connectivity strength')
# And add connect, source and brain objects to the scene
sc.add_to_subplot(s_obj)
sc.add_to_subplot(BrainObj('B3'), use_this_cam=True)

In [ ]:
# Coloring method
color_by = 'count'
# Weak connections -> alpha = .1 // strong connections -> alpha = 1.
dynamic = (.1, 1.)
# Define the connectivity and source object
c_count = ConnectObj('default', nodes, edges, select=select, line_width=4.,
                     color_by=color_by, antialias=True, dynamic=dynamic)
s_obj_c = SourceObj('sources', nodes, color='olive', radius_min=10.,
                    symbol='square')
# And add connect, source and brain objects to the scene
sc.add_to_subplot(c_count, row=0, col=1,
                  title='Color by number of connections per node')
sc.add_to_subplot(s_obj_c, use_this_cam=True, row=0, col=1)
sc.add_to_subplot(BrainObj('B3'), use_this_cam=True, row=0, col=1)

In [ ]:
# First, we take a copy of the connectivity array
edges_copy = edges.copy()
# Then, we force edges to take fixed values
# ====================   =========  ===========
# Condition              New value  Color
# ====================   =========  ===========
# edges >= 0.8              4.      red
# edges in [.78, .8[        3.      orange
# edges in [.74, .78[       2.      blue
# Others                    -       lightgray
# ====================   =========  ===========
edges_copy[edges_copy >= .8] = 4.
edges_copy[np.logical_and(edges_copy >= .78, edges_copy < .8)] = 3.
edges_copy[np.logical_and(edges_copy >= .74, edges_copy < .78)] = 2.
# Now we use a dctionary to set one color per value.
ccol = {
    None: 'lightgray',
    2.: 'blue',
    3.: 'orange',
    4.: 'red'
}

# Define the connectivity and source objects
c_cuscol = ConnectObj('default', nodes, edges_copy, select=edges > .7,
                      custom_colors=ccol)
s_obj_cu = SourceObj('sources', nodes, color='slategray', radius_min=10.,
                     symbol='ring')
# Add objects to the scene
sc.add_to_subplot(c_cuscol, row=0, col=2, title='Custom colors')
sc.add_to_subplot(s_obj_cu, row=0, col=2)
sc.add_to_subplot(BrainObj('white'), use_this_cam=True, row=0, col=2)

# Finally, display the scene
sc.preview()
sc.screenshot('/Users/gracer/Google Drive/ABCD/tmp/figlayout.png', dpi=600)

## Saving the data

In [ ]:
no_ROIs = list(noPART.keys())
no_MODS = list(noPART.values())

ov_ROIs = list(ovPART.keys())
ov_MODS = list(ovPART.values())

ob_ROIs = list(obPART.keys())
ob_MODS = list(obPART.values())


In [ ]:
nodf = pd.DataFrame(no_MODS)
nodf['ROI'] = no_ROIs
# nodf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/normal_Module.csv", sep=',', index=False)

In [ ]:
ovdf = pd.DataFrame(ov_MODS)
ovdf['ROI'] = ov_ROIs
# ovdf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/overweight_Module.csv", sep=',', index=False)

In [ ]:
obdf = pd.DataFrame(ob_MODS)
obdf['ROI'] = ob_ROIs
# obdf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/obese_Module.csv", sep=',', index=False)

In [ ]:
early_ROIs = list(earlyPART.keys())
early_MODS = list(earlyPART.values())

mid_ROIs = list(midPART.keys())
mid_MODS = list(midPART.values())

late_ROIs = list(latePART.keys())
late_MODS = list(latePART.values())


In [ ]:
earlydf = pd.DataFrame(early_MODS)
earlydf['ROI'] = early_ROIs
earlydf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/early_Module.csv", sep=',', index=False)

In [ ]:
middf = pd.DataFrame(mid_MODS)
middf['ROI'] = mid_ROIs
middf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/mid_Module.csv", sep=',', index=False)

In [ ]:
latedf = pd.DataFrame(late_MODS)
latedf['ROI'] = late_ROIs
latedf.to_csv("/Users/gracer/Google Drive/ABCD/tmp/late_Module.csv", sep=',', index=False)

## Parcelation 
Through BIAC https://wiki.biac.duke.edu/biac:analysis:resting_pipeline

## Individual and Group Matrices
Network-level analysis will be performed with inividual correltion matrices

## Thresholding
In accordance with van den Heuvel et al. 2017, we will examine and test statistical differences in functional connectivity (FC) defined as the mean of the correlation matrix. FC will be included in statistical tests between groups.

## Partitioning
Will partition full 264 connectome into modules using louvain algorithm. 

## Check the partition
Will use normalized mutual information to assess similarity between network assignments. NMI measures information shared between two probability distribution functions, specifically measuring how much knowing one distribution leads to certainty ofthe other. Permuted the labels of individual matrices between contrasts 1,000 times to generate a null distribution of NMI values for each contrast. Matrices between groups were randomly shuffled and partitioned into functional networks, and NMI was calculated.   
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.normalized_mutual_info_score.html#sklearn.metrics.normalized_mutual_info_score

## Connectivity Strength
Caluclate Euclidean distance for each ROI-ROI pair. Linear regression with distance as a predictor of connectivity strength between groups.

### Within network changes
All within network pairwise relationships were averaged per group. Two-tailed T-test to assess differences. Bonferroni corrections as needed.

### Between network changes
Average connectivity is calculated per network. Compare the between network interactions. 

## Participation Coefficient
Partition networks into the modules, calculate the PC per node within each group. Higher PC indicates more distributed between network connectivity, while a PC of 0 signifies a node’s links are completely within its home network (within network).

## Pickling data to save it

In [ ]:
# sfile = bz2.BZ2File('/Users/gracer/Google Drive/ABCD/tmp/smallerfile', 'w')
# pickle.dump(GRAPHS, sfile)
# GRAPHS['BIGdf'] = BIGdf
# GRAPHS['maxes'] = maxes
# pickle.dump(saveme, open('/Users/gracer/Google Drive/ABCD/tmp/just_mean', 'wb'), protocol=4)